# E5 (multilingual) — Retrieval Baseline, Fine-tuning (LoRA optional) on FrenchNews

Ce notebook montre un flux simple et reproductible :

1) Chargement du dataset `FrenchNews.csv` et création de paires (question / réponse).

2) Split en *train / validation / test*.

3) Encodage avec **E5** (`intfloat/multilingual-e5-small`) et évaluation (Recall@k, nDCG@k, MRR@k).

4) **Fine-tuning** léger avec Sentence-Transformers (perte MultipleNegativesRankingLoss), puis réévaluation.

5) **(Optionnel)** Fine-tuning via **LoRA** (PEFT) pour limiter le nombre de paramètres à mettre à jour.

> Objectif : garder la complexité basse, avec des blocs indépendants que vous pouvez exécuter l’un après l’autre.

## 0) Pré-requis & installation des dépendances
Exécutez ce bloc si votre environnement n’a pas déjà les bibliothèques nécessaires.

In [ ]:
!pip -q install --upgrade pip
!pip -q install torch --index-url https://download.pytorch.org/whl/cpu
!pip -q install sentence-transformers>=3.0.0 scikit-learn faiss-cpu pandas numpy tqdm datasets peft accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 9.8 MB/s eta 0:00:00


## 1) Configuration & Chargement du CSV
- Le chemin par défaut pointe vers le fichier fourni: `/mnt/data/FrenchNews.csv`.
- Nous utilisons **Titre** comme *question* et **Contenu** comme *réponse/contexte* (simple et efficace).

In [ ]:
import pandas as pd
from pathlib import Path

CSV_PATH = Path('FrenchNews.csv')  # Changez si nécessaire
df = pd.read_csv(CSV_PATH)
print(df.shape)
print(df.columns.tolist())
df[['Titre','Contenu']].head(5)

(41543, 20)
['Numero news', 'Numero page', 'Numero', 'Date', 'Heure', 'Titre', 'Contenu', 'Agency', 'URL', 'textURL', 'Nbr image', 'seconds to 2010', 'days to 2010', 'dateDT', 'Title eng', 'Content eng', 'textURL eng', 'Sentiment Vader Title', 'Sentiment Vader Text', 'Sentiment Vader TextURL']


,Titre,Contenu
0,Marseille : une baleine de 15 mètres piégée da...,C'est une drôle de découverte qu'ont faite ce ...
1,"Le Burkinabé qui a stoppé le désert, l'intox d...",L'émission de cette semaine nous emmène au Bur...
2,"En Grande-Bretagne, les ventes au détail subis...",LONDRES (Reuters) - Les ventes au détail en Gr...
3,Antiterrorisme : Bruxelles fait avec les moyen...,La sécurité ne fait pas partie des prérogative...
4,"Dmitri Rybolovlev, président de l'AS Monaco, i...","NICE (Reuters) - Le président de l'AS Monaco, ..."


In [ ]:
## 2) Préparation des paires (question / réponse)
# Nous filtrons les lignes valides et tronquons légèrement les textes trop longs pour l'entraînement rapide.

import numpy as np

df_pairs = df[['Titre','Contenu']].dropna().rename(columns={'Titre':'question','Contenu':'answer'}).copy()
# nettoyage très léger
df_pairs['question'] = df_pairs['question'].astype(str).str.strip()
df_pairs['answer'] = df_pairs['answer'].astype(str).str.strip()

# Retirez les lignes trop courtes
df_pairs = df_pairs[(df_pairs['question'].str.len() > 10) & (df_pairs['answer'].str.len() > 20)].reset_index(drop=True)
print('Paires valides:', len(df_pairs))
df_pairs.head(3)

Paires valides: 40976


,question,answer
0,Marseille : une baleine de 15 mètres piégée da...,C'est une drôle de découverte qu'ont faite ce ...
1,"Le Burkinabé qui a stoppé le désert, l'intox d...",L'émission de cette semaine nous emmène au Bur...
2,"En Grande-Bretagne, les ventes au détail subis...",LONDRES (Reuters) - Les ventes au détail en Gr...


In [ ]:
## 3) Split Train / Validation / Test

from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df_pairs, test_size=0.15, random_state=42, shuffle=True)
train_df, val_df  = train_test_split(train_df, test_size=0.15, random_state=42, shuffle=True)

len(train_df), len(val_df), len(test_df)

(29604, 5225, 6147)

In [ ]:
## 4) Baseline embeddings (E5) & Index FAISS

from sentence_transformers import SentenceTransformer
import numpy as np
import faiss
from tqdm.auto import tqdm

# Modèle léger et multilingue
EMB_MODEL_NAME = "intfloat/multilingual-e5-small"
model = SentenceTransformer(EMB_MODEL_NAME)

def e5_format_query(q):
    # E5 recommande le prefix "query: " pour les requêtes et "passage: " pour les passages
    return f"query: {q}"

def e5_format_passage(p):
    return f"passage: {p}"

# Build corpus (passages) depuis l'ensemble TEST pour évaluer la récupération
test_corpus = test_df['answer'].tolist()
test_queries = test_df['question'].tolist()

# Encode corpus
corpus_emb = model.encode([e5_format_passage(p) for p in test_corpus], batch_size=64, show_progress_bar=True, convert_to_numpy=True, normalize_embeddings=True)

# FAISS index
d = corpus_emb.shape[1]
index = faiss.IndexFlatIP(d)  # dot product with normalized vecs -> cosine similarity
index.add(corpus_emb)
index.ntotal

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

Batches:   0%|          | 0/97 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
## 5) Évaluation (Recall@k, MRR@k, nDCG@k)

def recall_at_k(truth, preds, k=10):
    # truth: int (gold index)
    # preds: list[int] predicted indices sorted by score desc
    return 1.0 if truth in preds[:k] else 0.0

def mrr_at_k(truth, preds, k=10):
    for i, pid in enumerate(preds[:k]):
        if pid == truth:
            return 1.0 / (i+1)
    return 0.0

def ndcg_at_k(truth, preds, k=10):
    # gain 1 for the relevant item at rank r
    for i, pid in enumerate(preds[:k]):
        if pid == truth:
            return 1.0 / np.log2(i+2)  # DCG with gain 1
    return 0.0  # no relevant item in top-k

def evaluate_retrieval(queries, corpus, k_list=[1,5,10]):
    q_emb = model.encode([e5_format_query(q) for q in queries], batch_size=64, show_progress_bar=True, convert_to_numpy=True, normalize_embeddings=True)
    scores, idxs = index.search(q_emb, max(k_list))
    out = {}
    for k in k_list:
        recs, mrrs, ndcgs = [], [], []
        for gold, pred_list in enumerate(idxs):
            # Ici, nous considérons la vérité comme l'item au même index (question i -> passage i)
            recs.append(recall_at_k(gold, pred_list.tolist(), k))
            mrrs.append(mrr_at_k(gold, pred_list.tolist(), k))
            ndcgs.append(ndcg_at_k(gold, pred_list.tolist(), k))
        out[f"Recall@{k}"] = float(np.mean(recs))
        out[f"MRR@{k}"]    = float(np.mean(mrrs))
        out[f"nDCG@{k}"]   = float(np.mean(ndcgs))
    return out

baseline_metrics = evaluate_retrieval(test_queries, test_corpus, k_list=[1,5,10])
baseline_metrics

In [ ]:
## 6) Fine-tuning simple (Sentence-Transformers) — sans LoRA

from sentence_transformers import InputExample, losses
from torch.utils.data import DataLoader

# Construire des paires positives (query, positive_passage) depuis le train
train_examples = [InputExample(texts=[f"query: {q}", f"passage: {a}"]) for q,a in zip(train_df['question'], train_df['answer'])]
val_examples   = [InputExample(texts=[f"query: {q}", f"passage: {a}"]) for q,a in zip(val_df['question'],   val_df['answer'])]

train_dataloader = DataLoader(train_examples, batch_size=64, shuffle=True, drop_last=True)
train_loss = losses.MultipleNegativesRankingLoss(model)

# Entraînement court pour la démonstration
from sentence_transformers import SentenceTransformer
num_epochs = 1
warmup_steps = int(len(train_dataloader) * num_epochs * 0.1)

model_save_path = "e5-small-finetuned-simple"
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=num_epochs,
    warmup_steps=warmup_steps,
    use_amp=True,
    output_path=model_save_path
)

# Recharger le modèle finetuné
ft_model = SentenceTransformer(model_save_path)

# Réindexer le corpus test avec le modèle finetuné
ft_corpus_emb = ft_model.encode([f"passage: {p}" for p in test_corpus], batch_size=64, show_progress_bar=True, convert_to_numpy=True, normalize_embeddings=True)
import faiss
index_ft = faiss.IndexFlatIP(ft_corpus_emb.shape[1])
index_ft.add(ft_corpus_emb)

def evaluate_with_model(_model, _index, queries, k_list=[1,5,10]):
    q_emb = _model.encode([f"query: {q}" for q in queries], batch_size=64, show_progress_bar=True, convert_to_numpy=True, normalize_embeddings=True)
    scores, idxs = _index.search(q_emb, max(k_list))
    out = {}
    import numpy as np
    for k in k_list:
        recs, mrrs, ndcgs = [], [], []
        for gold, pred_list in enumerate(idxs):
            pred_list = pred_list.tolist()
            recs.append(1.0 if gold in pred_list[:k] else 0.0)
            # MRR
            rr = 0.0
            for i, pid in enumerate(pred_list[:k]):
                if pid == gold:
                    rr = 1.0 / (i+1)
                    break
            mrrs.append(rr)
            # nDCG
            dcg = 0.0
            for i, pid in enumerate(pred_list[:k]):
                if pid == gold:
                    dcg = 1.0 / np.log2(i+2)
                    break
            ndcgs.append(dcg)
        out[f"Recall@{k}"] = float(np.mean(recs))
        out[f"MRR@{k}"]    = float(np.mean(mrrs))
        out[f"nDCG@{k}"]   = float(np.mean(ndcgs))
    return out

ft_metrics = evaluate_with_model(ft_model, index_ft, test_queries, k_list=[1,5,10])
ft_metrics

In [ ]:
## 7) (Optionnel) Fine-tuning via LoRA (PEFT)

# Remarque: Cette section suppose des versions récentes de sentence-transformers (>=3.0) et peft.
# On applique LoRA sur le backbone Transformer du modèle E5 pour réduire le nb de paramètres entraînables.

from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from sentence_transformers import SentenceTransformer

lora_model = SentenceTransformer(EMB_MODEL_NAME)

# Récupérer le backbone AutoModel pour appliquer PEFT
backbone = lora_model._first_module().auto_model  # partie Transformer interne
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="FEATURE_EXTRACTION",  # approprié pour embeddings
)

peft_backbone = get_peft_model(backbone, lora_config)
# Reconnecter dans Sentence-Transformers
lora_model._first_module().auto_model = peft_backbone

# Préparer data loader (on réutilise train_examples)
from torch.utils.data import DataLoader
from sentence_transformers import losses

train_dataloader_lora = DataLoader(train_examples, batch_size=64, shuffle=True, drop_last=True)
train_loss_lora = losses.MultipleNegativesRankingLoss(lora_model)

num_epochs = 1
warmup_steps = int(len(train_dataloader_lora) * num_epochs * 0.1)
lora_output = "e5-small-finetuned-lora"

lora_model.fit(
    train_objectives=[(train_dataloader_lora, train_loss_lora)],
    epochs=num_epochs,
    warmup_steps=warmup_steps,
    use_amp=True,
    output_path=lora_output
)

# Évaluation post-LoRA
lora_model = SentenceTransformer(lora_output)
lora_corpus_emb = lora_model.encode([f"passage: {p}" for p in test_corpus], batch_size=64, show_progress_bar=True, convert_to_numpy=True, normalize_embeddings=True)
index_lora = faiss.IndexFlatIP(lora_corpus_emb.shape[1])
index_lora.add(lora_corpus_emb)

lora_metrics = evaluate_with_model(lora_model, index_lora, test_queries, k_list=[1,5,10])
lora_metrics

In [ ]:
## 8) Comparaison des métriques

import pandas as pd

rows = [ {"model":"E5 baseline", **baseline_metrics},
         {"model":"E5 finetuned (simple)", **ft_metrics} ]

try:
    rows.append({"model":"E5 finetuned (LoRA)", **lora_metrics})
except NameError:
    pass

pd.DataFrame(rows)

In [ ]:
## (Annexe) Sauvegarder des jeux de données structurés

train_df.to_csv("train_pairs.csv", index=False)
val_df.to_csv("val_pairs.csv", index=False)
test_df.to_csv("test_pairs.csv", index=False)
print("Fichiers écrits: train_pairs.csv, val_pairs.csv, test_pairs.csv")

> Astuce: si vous disposez d’un GPU, changez l’installation de torch pour la version CUDA et augmentez `num_epochs`.